In [1]:
from libraries import lib
from libraries.NeuralStates import *
import torch
import torch.nn as nn
import time
from kan import *

In [2]:
def generate_eloc_distr(sampled_vector, N, J, Gamma, model):
    f_start = time.time()

    times = {'dict':0, 'gen state':0, 'forward':0, 'model to output':0, 'gen adj':0, 'calc h elem':0, 'calc exp':0, 'multiply':0}

    nn_output_calcs = {}
    def model_to_output(x):
        start = time.time()
        if x in sampled_vector.nn_output:
            times['dict'] += time.time() - start
            return sampled_vector.nn_output[x]
        if x in nn_output_calcs:
            times['dict'] += time.time() - start
            return nn_output_calcs[x]
        start = time.time()
        tens = torch.tensor([lib.generate_state_array(x, N)], dtype = torch.float32)
        times['gen state'] += time.time() - start

        start = time.time()
        output = model(tens)[0]
        times['forward'] += time.time() - start
        nn_output_calcs[x] = output
        return output
    
    eloc_values = {}
    for basis_state in sampled_vector.distribution:
        eloc = 0
        start = time.time() 
        output = model_to_output(basis_state).detach()
        # print(output)
        times['model to output'] += time.time() - start
        start = time.time()
        adj = lib.generate_adjacencies(basis_state, N)
        times['gen adj'] += time.time() - start
        for adjacency in adj:
            start = time.time()
            output_prime = model_to_output(adjacency).detach()
            times['model to output'] += time.time() - start
            start = time.time()
            h_elem = lib.calc_H_elem(N, J, Gamma, basis_state, adjacency)
            times['calc h elem'] += time.time() - start
            start = time.time()
            exp = torch.exp(output_prime[0] - output[0] + 1.j * (output_prime[1] - output[1]))
            times['calc exp'] += time.time() - start
            start = time.time()
            eloc += h_elem * exp
            times['multiply'] += time.time() - start
        eloc_values[basis_state] = eloc
    for k in times:
        print(f"{k}: {times[k]}")
    print('total', time.time() - f_start)
    return eloc_values

In [3]:
def log_amp_phase(nn_output):
    return torch.exp(nn_output[:, 0] + 1.j * nn_output[:, 1])

def bitflip_x(x, N, flips):
    new_x = x
    for _ in range(flips):
        new_x = x ^ (1 << npr.randint(0, N))
    return new_x

In [4]:
N = 30; J = 1; h=10

In [5]:
layers = []
layers.append(nn.Linear(N, 32))
for _ in range(2):
    layers.append(nn.Linear(32, 32))
    layers.append(nn.SELU())
layers.append(nn.Linear(32, 2))
mlp_model = nn.Sequential(*layers)

In [6]:
# %%timeit
num_samples = 512
mh_state = MHNeuralState(N, mlp_model, log_amp_phase, lambda x : bitflip_x(x, N, 1), 0, num_samples)

In [7]:
# %%timeit
eloc_list = generate_eloc_distr(mh_state, N, J, h, mlp_model)
# for small n values, the calc exp step is most significant, for larger n values the forward passes are most significant

dict: 0.0010027885437011719
gen state: 0.09887480735778809
forward: 0.6763584613800049
model to output: 0.814948558807373
gen adj: 0.0034945011138916016
calc h elem: 0.05569767951965332
calc exp: 0.3032097816467285
multiply: 0.06017804145812988
total 1.2427425384521484


In [8]:
N=10
kan_model = KAN(width=[N, N, 2], device='cpu', seed=0, auto_save=False);

In [9]:
num_samples = 512
kan_mh_state = MHNeuralState(N, kan_model, log_amp_phase, lambda x : bitflip_x(x, N, 1), 0, num_samples)

c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (inpu

In [10]:
eloc_list = generate_eloc_distr(kan_mh_state, N, J, h, kan_model)
# for small n values, the calc exp step is most significant, for larger n values the forward passes are most significant (same as MLP)
# however it seems that forward time increases much faster as N increases compared to MLP (may be parameter scaling)
# N=10 for KAN has forward super dominant while N=30 for MLP has 2:1 forward to exp ratio

dict: 0.0009980201721191406
gen state: 0.014627218246459961
forward: 5.3628990650177
model to output: 5.38679313659668
gen adj: 0.0
calc h elem: 0.01102304458618164
calc exp: 0.08408975601196289
multiply: 0.018344402313232422
total 5.502248764038086


In [11]:
def generate_input_samples(N, samples):
    return torch.tensor([lib.generate_state_array(x, N) for x in samples]).to(torch.float32)
nn_output_calcs = {}
def model_to_output(x, sampled_vector, model):
    if x in sampled_vector.nn_output:
        return sampled_vector.nn_output[x]
    if x in nn_output_calcs:
        return nn_output_calcs[x]
    tens = torch.tensor([lib.generate_state_array(x, N)], dtype = torch.float32)
    output = model(tens)[0]
    nn_output_calcs[x] = output
    return output
def generate_eloc_distr_no_time(sampled_vector, N, J, Gamma, model):
    eloc_values = {}
    for basis_state in sampled_vector.distribution:
        eloc = 0
        output = model_to_output(basis_state, sampled_vector, model)
        for adjacency in lib.generate_adjacencies(basis_state, N):
            output_prime = model_to_output(adjacency)
            eloc += lib.calc_H_elem(N, J, Gamma, basis_state, adjacency) * torch.exp(output_prime[0] - output[0] + 1.j * (output_prime[1] - output[1]))
        eloc_values[basis_state] = eloc
    return eloc_values

In [12]:
N = 10
states = [n for n in range(2 ** N)]
kan_model = KAN([N, N, 2])
kan_mh_state = MHNeuralState(N, kan_model, log_amp_phase, lambda x : bitflip_x(x, N, 1), 0, 0)

checkpoint directory created: ./model
saving model version 0.0


In [13]:
start = time.time()
nn_output_calcs = {}
for s in states:
    model_to_output(s, kan_mh_state, kan_model)
print(f'KAN unbacthced time: {time.time() - start}')
start = time.time()
batch = generate_input_samples(N, states)
kan_model(batch)
print(f'KAN batched time {time.time() - start}')

KAN unbacthced time: 8.943184614181519
KAN batched time 0.026005268096923828


In [14]:
layers = []
layers.append(nn.Linear(N, 32))
for _ in range(2):
    layers.append(nn.Linear(32, 32))
    layers.append(nn.SELU())
layers.append(nn.Linear(32, 2))
mlp_model = nn.Sequential(*layers)
mlp_mh_state = MHNeuralState(N, mlp_model, log_amp_phase, lambda x : bitflip_x(x, N, 1), 0, 0)

In [15]:
start = time.time()
nn_output_calcs = {}
for s in states:
    model_to_output(s, mlp_mh_state, mlp_model)
print(f'MLP unbacthced time: {time.time() - start}')
start = time.time()
batch = generate_input_samples(N, states)
mlp_model(batch)
print(f'MLP batched time {time.time() - start}')

MLP unbacthced time: 1.6340570449829102
MLP batched time 0.003298044204711914


In [16]:
batch1 = generate_input_samples(N, [1])
batch2 = generate_input_samples(N, [1, 2, 3])

In [17]:
%%timeit
mlp_model(batch1)

34.6 μs ± 1.35 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
%%timeit
mlp_model(batch2)

36.2 μs ± 1.55 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
%%timeit
kan_model(batch1)

c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
c:\Users\taoha\anaconda3\envs\kans\Lib\site-packages\kan\MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (inpu

8.1 ms ± 191 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
kan_model(batch2)

7.3 ms ± 101 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


: 